In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import matplotlib.pyplot as plt 
%matplotlib inline
from subprocess import check_output
print(check_output(["ls", "input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

members.csv
sample_submission.csv
song_extra_info.csv
songs.csv
test.csv
train.csv



# Feature extraction and cleaning

In [2]:
df_train = pd.read_csv("input/train.csv")

1


In [4]:
df_songs = pd.read_csv("input/songs.csv")

In [5]:
df_songs_extra = pd.read_csv("input/song_extra_info.csv")

df_members = pd.read_csv("input/members.csv",parse_dates=["registration_init_time","expiration_date"])

In [6]:
df_test = pd.read_csv("input/test.csv")

In [7]:
df_train =df_train.merge(df_songs,how="left",on="song_id")

In [8]:
df_train = df_train.merge(df_members,how="left",on="msno")

In [9]:
df_test =df_test.merge(df_songs,how="left",on="song_id")

In [10]:
df_test = df_test.merge(df_members,how="left",on="msno")

**Handling Null Values **

In [11]:
df_train['gender'].fillna(value="Unknown",inplace=True)
df_test['gender'].fillna(value="Unknown",inplace=True)

In [12]:
df_train['source_system_tab'].fillna(value="Unknown",inplace=True)
df_test['source_system_tab'].fillna(value="Unknown",inplace=True)


df_train['source_screen_name'].fillna(value="Unknown",inplace=True)
df_test['source_screen_name'].fillna(value="Unknown",inplace=True)

df_train['source_type'].fillna(value="Unknown",inplace=True)
df_test['source_type'].fillna(value="Unknown",inplace=True)

df_train['genre_ids'].fillna(value="Unknown",inplace=True)
df_test['genre_ids'].fillna(value="Unknown",inplace=True)

df_train['artist_name'].fillna(value="Unknown",inplace=True)
df_test['artist_name'].fillna(value="Unknown",inplace=True)

df_train['composer'].fillna(value="Unknown",inplace=True)
df_test['composer'].fillna(value="Unknown",inplace=True)

df_train['lyricist'].fillna(value="Unknown",inplace=True)
df_test['lyricist'].fillna(value="Unknown",inplace=True)

In [13]:
df_train['song_length'].fillna(value=df_train['song_length'].mean(),inplace=True)

df_test['song_length'].fillna(value=df_test['song_length'].mean(),inplace=True)

In [14]:
df_train['language'].fillna(value=df_train['language'].mode()[0],inplace=True)

df_test['language'].fillna(value=df_test['language'].mode()[0],inplace=True)

In [15]:
gid_list = df_train['genre_ids'].unique()
gid_dic = {}
for i in range(len(gid_list)):
    gid_dic[gid_list[i]] = i
df_train['genre_ids_new'] = df_train['genre_ids'].map(gid_dic)
df_test['genre_ids_new'] = df_test['genre_ids']
df_test["genre_ids_new"] = df_test["genre_ids_new"].apply(lambda x : x if x in gid_dic else "Unknown").map(gid_dic)

**Genre IDS**

In [18]:
df_train['genre_ids'] = df_train['genre_ids'].str.split("|")

df_test['genre_ids'] = df_test['genre_ids'].str.split("|")

In [20]:
df_train['genre_count'] = df_train['genre_ids'].apply(lambda x : len(x) if "Unknown" not in x else 0)

df_test['genre_count'] = df_test['genre_ids'].apply(lambda x : len(x) if "Unknown" not in x else 0)

**Artists **

In [22]:
df_train['artist_name'].nunique()

40583

In [23]:
df_test['artist_name'].nunique()

27564

In [24]:
print("Number of Common Artists in both train & Test : ", len(set.intersection(set(df_train['artist_name']), set(df_test['artist_name']))))

Number of Common Artists in both train & Test :  21774


In [25]:
df_artists = df_train.loc[:,["artist_name","target"]]

artists1 = df_artists.groupby(["artist_name"],as_index=False).sum().rename(columns={"target":"repeat_count"})

artists2 = df_artists.groupby(["artist_name"],as_index=False).count().rename(columns = {"target":"play_count"})

In [26]:
artists1.head()

,artist_name,repeat_count
0,!Conz,1
1,!magnic!,0
2,#037 (#037),20
3,#1 Beatles Now,7
4,#1 Club Hits - Best Of Dance & Techno,5


In [27]:
df_artist_repeats = artists1.merge(artists2,how="inner",on="artist_name")

In [28]:
df_artist_repeats.head()

,artist_name,repeat_count,play_count
0,!Conz,1,1
1,!magnic!,0,1
2,#037 (#037),20,84
3,#1 Beatles Now,7,32
4,#1 Club Hits - Best Of Dance & Techno,5,7


In [29]:
df_artist_repeats["repeat_percentage"] = round((df_artist_repeats['repeat_count']*100) / df_artist_repeats['play_count'],1)

In [30]:
df_artist_repeats.head()

,artist_name,repeat_count,play_count,repeat_percentage
0,!Conz,1,1,100.0
1,!magnic!,0,1,0.0
2,#037 (#037),20,84,23.8
3,#1 Beatles Now,7,32,21.9
4,#1 Club Hits - Best Of Dance & Techno,5,7,71.4


In [31]:
df_artist_repeats.head()

,artist_name,repeat_count,play_count,repeat_percentage
0,!Conz,1,1,100.0
1,!magnic!,0,1,0.0
2,#037 (#037),20,84,23.8
3,#1 Beatles Now,7,32,21.9
4,#1 Club Hits - Best Of Dance & Techno,5,7,71.4


In [32]:
df_artist_repeats.drop(["repeat_count","play_count"],axis=1,inplace=True)

In [33]:
df_train = df_train.merge(df_artist_repeats,on="artist_name",how="left").rename(columns={"repeat_percentage":"artist_repeat_percentage"})

In [34]:
df_test = df_test.merge(df_artist_repeats,on="artist_name",how="left").rename(columns={"repeat_percentage":"artist_repeat_percentage"})

In [35]:
df_train.drop(["genre_ids","artist_name"],axis=1,inplace=True)

df_test.drop(["genre_ids","artist_name"],axis=1,inplace=True)

In [36]:
del df_artist_repeats
del df_artists

**Composers**

In [37]:
df_train['composer'] = df_train['composer'].str.split("|") 

df_test['composer'] = df_test['composer'].str.split("|") 

In [38]:
df_train['composer_count'] = df_train['composer'].apply(lambda x : len(x) if "Unknown" not in x else 0 )

df_test['composer_count'] = df_test['composer'].apply(lambda x : len(x) if "Unknown" not in x else 0 )

In [39]:
df_train['source_system_tab'].value_counts()

my library      3684730
discover        2179252
search           623286
radio            476701
listen with      212266
explore          167949
Unknown           24849
notification       6185
settings           2200
Name: source_system_tab, dtype: int64

In [40]:
source_tab_dict = {"my library":8,"discover":7,"search":6,"radio":5,"listen with":4,"explore":3,"notification":2,"settings":1,"Unknown":0 }

source_screen_name_dict = {"Local playlist more":19,"Online playlist more":18,"Radio":17,"Unknown":16,"Album more":15,"Search":14,"Artist more":13,"Discover Feature":12,"Discover Chart":11,"Others profile more":10,"Discover Genre":9,"My library":8,"Explore":7,"Discover New":6,"Search Trends":5,"Search Home":4,"My library_Search":3,"Self profile more":2,"Concert":1,"Payment":0}

source_type_dict = {"local-library":12,"online-playlist":11,"local-playlist":10,"radio":9,"album":8,"top-hits-for-artist":7,"song":6,"song-based-playlist":5,"listen-with":4,"Unknown":3,"topic-article-playlist":2,"artist":1,"my-daily-playlist":0}


In [41]:
df_train['source_system_tab'] = df_train['source_system_tab'].map(source_tab_dict)

df_test['source_system_tab'] = df_test['source_system_tab'].map(source_tab_dict)

In [42]:
df_train['source_type'] = df_train['source_type'].map(source_type_dict)
df_test['source_type'] = df_test['source_type'].map(source_type_dict)

In [43]:
df_train['source_screen_name'] = df_train['source_screen_name'].map(source_screen_name_dict)
df_test['source_screen_name'] = df_test['source_screen_name'].map(source_screen_name_dict)

In [44]:
gender_train = pd.get_dummies(df_train['gender'],drop_first=True)

gender_test = pd.get_dummies(df_test['gender'],drop_first=True)

In [45]:
df_train = pd.concat([df_train,gender_train],axis=1)
df_test = pd.concat([df_test,gender_test],axis=1)

In [46]:
df_train.drop(["composer","gender"],axis=1,inplace=True)
df_test.drop(["composer","gender"],axis=1,inplace=True)

In [47]:
df_train['validity_days'] = (df_train['expiration_date'] - df_train['registration_init_time']).dt.days

df_test['validity_days'] = (df_test['expiration_date'] - df_test['registration_init_time']).dt.days

In [48]:
df_train.drop(["registration_init_time","expiration_date"],axis=1,inplace=True)

df_test.drop(["registration_init_time","expiration_date"],axis=1,inplace=True)

In [49]:
df_train['lyricist'] = df_train['lyricist'].str.split("|")
df_test['lyricist'] = df_test['lyricist'].str.split("|")

In [50]:
df_train['lyricist_count'] = df_train['lyricist'].apply(lambda x : len(x) if "Unknown" not in x else 0 )

df_test['lyricist_count'] = df_test['lyricist'].apply(lambda x : len(x) if "Unknown" not in x else 0 )

In [51]:
df_test['artist_repeat_percentage'].fillna(value=0.0,inplace=True)

In [52]:
df_test['source_screen_name'].fillna(df_test['source_screen_name'].mode()[0],inplace=True)

In [53]:
df_train.drop("lyricist",axis=1,inplace=True)

df_test.drop("lyricist",axis=1,inplace=True)

In [54]:
df_songs_extra.drop("name",axis=1,inplace=True)

In [55]:
df_train = df_train.merge(df_songs_extra,how="left",on="song_id")

In [56]:
df_test = df_test.merge(df_songs_extra,how="left",on="song_id")

In [57]:
def isrc_to_year(isrc):
    if type(isrc) == str:
        if int(isrc[5:7]) > 17:
            return 1900 + int(isrc[5:7])
        else:
            return 2000 + int(isrc[5:7])
    else:
        return np.nan

In [58]:
df_train['song_year'] = df_train['isrc'].apply(isrc_to_year)

df_test['song_year'] = df_test['isrc'].apply(isrc_to_year)

In [59]:
df_train.drop("isrc",axis=1,inplace=True)

df_test.drop("isrc",axis=1,inplace=True)

In [60]:
df_train['song_year'].fillna(value=-1,inplace=True)

df_test['song_year'].fillna(value=-1,inplace=True)

In [61]:
df_train['song_year'] = df_train['song_year'].astype("int")

df_test['song_year'] = df_test['song_year'].astype("int")

In [62]:
df_train['source_system_tab'] = df_train['source_system_tab'].astype("category")
df_test['source_system_tab'] = df_test['source_system_tab'].astype("category")

df_train['source_screen_name'] = df_train['source_screen_name'].astype("category")
df_test['source_screen_name'] = df_test['source_screen_name'].astype("category")

df_train['source_type'] = df_train['source_type'].astype("category")
df_test['source_type'] = df_test['source_type'].astype("category")

df_train['language'] = df_train['language'].astype("category")
df_test['language'] = df_test['language'].astype("category")

df_train['city'] = df_train['city'].astype("category")
df_test['city'] = df_test['city'].astype("category")


df_train['registered_via'] = df_train['registered_via'].astype("category")
df_test['registered_via'] = df_test['registered_via'].astype("category")


In [63]:
X = df_train.drop(["msno","song_id","target","genre_count"],axis=1).values

y = df_train['target'].values
X_train = X[:1000000]
y_train = y[:1000000]
X_valid = X[1000000:]
y_valid = y[1000000:]

# Training the GBDT model

In [64]:
import lightgbm as lgb

d_train = lgb.Dataset(X_train, y_train)
d_valid = lgb.Dataset(X_valid,y_valid)
watchlist = [d_valid]
d = lgb.Dataset(X,y)

In [65]:
params = {}
params['learning_rate'] = 0.6
params['application'] = 'binary'
params['max_depth'] = 20
params['num_leaves'] = 2**5
params['verbosity'] = 0
params['metric'] = 'auc'
#params['boosting_type']= 'gbdt'

In [66]:
#model = lgb.train(params, train_set=d, num_boost_round=60, valid_sets=watchlist, \
#verbose_eval=5)

In [67]:
model = lgb.train(params, train_set=d, num_boost_round=60, valid_sets=watchlist, \
verbose_eval=5)
from sklearn.externals import joblib
joblib.dump(model, 'lgb.pkl')
model = joblib.load('lgb.pkl')

[5]	valid_0's auc: 0.674108
[10]	valid_0's auc: 0.679197
[15]	valid_0's auc: 0.681873
[20]	valid_0's auc: 0.685114
[25]	valid_0's auc: 0.687648
[30]	valid_0's auc: 0.689203
[35]	valid_0's auc: 0.691205
[40]	valid_0's auc: 0.692892
[45]	valid_0's auc: 0.694716
[50]	valid_0's auc: 0.696484
[55]	valid_0's auc: 0.697765
[60]	valid_0's auc: 0.69903


Mode was saved for future usage

In [68]:
song_ids = df_test['id'].values

X_test = df_test.drop(["msno","song_id","id","genre_count"],axis=1).values

In [73]:
model.feature_importance()

array([ 98, 112, 147,  97,  51, 183, 265,  82,  40, 172,  18,  35,  27,
       432,  20,  81])

In [74]:
df_train.drop(["msno","song_id","target"],axis=1).head()

,source_system_tab,source_screen_name,source_type,song_length,language,city,bd,registered_via,genre_ids_new,genre_count,artist_repeat_percentage,composer_count,female,male,validity_days,lyricist_count,song_year
0,3,7,11,206471.0,52.0,1,0,7,0,1,46.3,2,0,0,2103,0,2016
1,8,19,10,284584.0,52.0,13,24,9,1,1,51.0,0,1,0,2301,0,1999
2,8,19,10,225396.0,52.0,13,24,9,1,1,21.5,1,1,0,2301,0,2006
3,8,19,10,255512.0,-1.0,13,24,9,2,1,100.0,1,1,0,2301,0,2010
4,3,7,11,187802.0,52.0,1,0,7,3,1,37.7,3,0,0,2103,0,2016


In [75]:
"""from sklearn.linear_model import LogisticRegression
seed = 23
lr = LogisticRegression(max_iter=100,random_state=seed)
lr.fit(trainfeature,y)
result_df = pd.DataFrame()
predict_log = lr.predict_proba(testfeature)
result_df['id'] = song_ids
result_df['target'] = predict_log[:,1]
result_df.to_csv('predict_log.csv.gz', compression = 'gzip', index=False, float_format = '%.5f')
"""

"from sklearn.linear_model import LogisticRegression\nseed = 23\nlr = LogisticRegression(max_iter=100,random_state=seed)\nlr.fit(trainfeature,y)\nresult_df = pd.DataFrame()\npredict_log = lr.predict_proba(testfeature)\nresult_df['id'] = song_ids\nresult_df['target'] = predict_log[:,1]\nresult_df.to_csv('predict_log.csv.gz', compression = 'gzip', index=False, float_format = '%.5f')\n"

In [76]:
y_preds = model.predict(X_test)
result_df = pd.DataFrame()

result_df['id'] = song_ids
result_df['target'] = y_preds
result_df.to_csv('submission5.csv.gz', compression = 'gzip', index=False, float_format = '%.5f')

In [78]:
df_train.drop(["msno","song_id","genre_count"],axis=1).to_csv('tmp/train.csv')
df_test.drop(["msno","song_id","id","genre_count"],axis=1).to_csv('tmp/test.csv')

# Run a test using sklean package

In [79]:

def get_batch(X, y=None, batch_size = 100,feature_size = 2**5):
        """
        Return minibatch of samples and labels
        
        :param X, y: samples and corresponding labels
        :parma batch_size: minibatch size
        :returns: (tuple) X_batch, y_batch
        """
        def one_hot(y):
            return np.eye(feature_size)[y].squeeze()
        N,_ = X.shape 
        X_batch = []
        y_batch = []
        for i in range(0,N,batch_size):            
            x_ =  X[i:i+batch_size,:]
            x_one_hot = []
            
            for j in range(60):
                one = one_hot(x_[:,j])
                x_one_hot.append(one)
            if not y is None:
                yield np.concatenate(x_one_hot,axis = 1),y[i:i+batch_size]
            else:
                yield np.concatenate(x_one_hot,axis = 1),None


In [77]:
batchIter= get_batch(trainfeature,y)

In [78]:
from sklearn.linear_model import SGDClassifier
model = SGDClassifier(loss = 'log',penalty="l1")
def train(model,epoch):
    
    e = 0
    i = 0
    while(e < epoch):
        batchIter = get_batch(trainfeature,y)
        for x_,y_  in batchIter:
            model.partial_fit(x_,y_,classes = [0,1])
            i += 1
            if i % 1000 == 0:
                print(i,"iter done")
                print(model.score(x_,y_))
        print(e, 'epoch done')
        e += 1
    return model

In [79]:
train(model,3)

/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


1000 iter done
0.82
2000 iter done
0.85
3000 iter done
0.77
4000 iter done
0.85
5000 iter done
0.87
6000 iter done
0.84
7000 iter done
0.8
8000 iter done
0.83
9000 iter done
0.79
10000 iter done
0.74
11000 iter done
0.64
12000 iter done
0.76
13000 iter done
0.69
14000 iter done
0.78
15000 iter done
0.85
16000 iter done
0.9
17000 iter done
0.74
18000 iter done
0.78
19000 iter done
0.62
20000 iter done
0.79
21000 iter done
0.63
22000 iter done
0.68
23000 iter done
0.7
24000 iter done
0.6
25000 iter done
0.67
26000 iter done
0.71
27000 iter done
0.72
28000 iter done
0.72
29000 iter done
0.65
30000 iter done
0.63
31000 iter done
0.66
32000 iter done
0.65
33000 iter done
0.69
34000 iter done
0.63
35000 iter done
0.75
36000 iter done
0.55
37000 iter done
0.68
38000 iter done
0.65
39000 iter done
0.73
40000 iter done
0.57
41000 iter done
0.77
42000 iter done
0.6
43000 iter done
0.77
44000 iter done
0.7
45000 iter done
0.65
46000 iter done
0.67
47000 iter done
0.57
48000 iter done
0.66
49000 i

KeyboardInterrupt: 

In [ ]:
#

In [ ]:
#y_logistic = y.predict_proba()

In [ ]:
x_,y_ = next(batchIter)
print(x_,y_)

In [80]:
def test(model,testfeature):
    #model = SGDClassifier(loss = 'log')
    y = []
    batchIter = get_batch(testfeature)
    for x_,_  in batchIter:
        y_ = model.predict_proba(x_)
        y.append(y_)   
    return np.concatenate(y)
y_logistic = test(model,testfeature)
y_logistic = y_logistic[:,1]

In [81]:
result_df = pd.DataFrame()

result_df['id'] = song_ids
result_df['target'] = y_logistic
result_df.to_csv('gbdt_logistic.csv.gz', compression = 'gzip', index=False, float_format = '%.5f')

In [ ]:
#test(model,trainfeature[:10,:])

In [ ]:
y[:100]

In [83]:
y_logistic

array([ 0.57749766,  0.58682825,  0.19088878, ...,  0.30009641,
        0.00766435,  0.00774449])